# **LoopsResultsAnalysis**

## **1. Importing and First Proccesing**

In [1]:
import pandas as pd
from scipy import stats
import plotly.express as px
import ProcessingConfig as config
import ProcessingFuncs as process
import AnalysisFuncs as analyze

pd.options.display.max_columns = 50

In [2]:
raw_data = pd.read_excel(config.cleaning_config['raw_data_path'])

first_lines_data = process.clean_data(raw_data, only_first_lines=True)
process.save_in_excel(first_lines_data)

analyze.get_sample_size(first_lines_data)

original shape: (19701, 32)
threshold for outliers detection: 2.25
filter_slow_subjects: No slow subjects detected.
filter_bad_subjects: No bad subjects detected (in terms of low success rate).
drop_first_loop: 2419 rows were filtered out.
filter_bad_trials: No bad trials detected (in terms of low success rate).
filter_slow_steps: 99 slow steps were filtered out.
Here is a summary of slow steps rate per subjects: 
     subject              101A  110A  111A  104A  111B  106A  106B  102A  112A  \
slow steps rate (%)   0.0   0.0  0.64  0.64  1.19  1.96  1.96  1.96  2.38   

subject              102B  103A  101B  105B  105A  107A  109B  108A  110B  \
slow steps rate (%)  2.58   2.7  3.29  3.31  3.42  3.57  3.87   3.9  3.97   

subject              107B  104B  108B  109A  103B  
slow steps rate (%)   4.0  5.11  5.48  5.48  7.09  
final shape: (3514, 26)
There are 43 sessions from 23 subjects


## **2. General Priming Effect Testing**

In [3]:
# mean response time and success rate, group by loop type and loop switching.
switching_diff_basic = pd.DataFrame()

switching_diff_basic['mean_response_time'] = first_lines_data[first_lines_data['correct']].groupby(['loop_type', 'loop_type_switch'])['rt'].mean()

switching_diff_basic

mean_response_time
loop_type loop_type_switch                    
for       False                    3219.194896
          True                     3236.056537
while     False                    3493.503597
          True                     3522.751493

In [4]:
# checking for a significant pearson correlation between switching and response time
analyze.test_rt_switch_corr(first_lines_data)

Pearson correlation between loop type switching and response time 
          is not significant (p = 0.425), with value of r = 0.013


## **3. Different Influencors on Priming Effect**

#### **3.1. Mistake in Previous Step**

In [5]:
indexed_raw_data = raw_data.set_index(['subject', 'trial', 'step_num'])
indexed_raw_data = indexed_raw_data[['correct', 'text1']]

def is_prev_correct(step):
    subject = step.loc['subject']
    step_num = step.loc['step_num']
    trial = step.loc['trial']
    
    if step_num - 1 < 0:
        trial -= 1
        step_num = max(indexed_raw_data.loc[(subject, trial), 'step_num'])
    
    return indexed_raw_data.loc[(subject, trial, step_num - 1), 'correct']

filtered_data['is_prev_correct'] = filtered_data.apply(is_prev_correct, axis=1)

NameError: name 'filtered_data' is not defined

In [ ]:
# mean response time and success rate, group by loop type and loop switching.
switching_diff_prev_correct = pd.DataFrame()

switching_diff_prev_correct['mean_response_time'] = filtered_data[filtered_data['correct']].groupby(['loop_type', 'loop_type_switch', 'is_prev_correct'])['rt'].mean()
switching_diff_prev_correct['number_of_steps'] = filtered_data[filtered_data['correct']].groupby(['loop_type', 'loop_type_switch', 'is_prev_correct'])['step_num'].count()

switching_diff_prev_correct

mean_response_time  \
loop_type loop_type_switch is_prev_correct                       
for       False            False                   2295.500000   
                           True                    3221.343023   
          True             False                   1928.000000   
                           True                    3239.145218   
while     False            False                   1692.000000   
                           True                    3495.666267   
          True             False                   3463.000000   
                           True                    3522.822967   

                                            number_of_steps  
loop_type loop_type_switch is_prev_correct                   
for       False            False                          2  
                           True                         860  
          True             False                          2  
                           True                         847  
while     False            False                          1  
                           True                         833  
          True             False                          1  
                           True                         836